In [1]:
import numpy as np
from scipy.linalg import cholesky, cho_solve

In [2]:
from bake.kernels import s_gaussian

In [3]:
np.random.seed(100)
np.set_printoptions(2)
n = 2000
m = 10
d = 7 * 7 * 64
n_q = 10
x = np.random.rand(n, d)
x_q = np.random.rand(n_q, d)
y = np.random.randint(0, m, (n, 1))
classes = np.unique(y)
y_one_hot = y == classes
g = y_one_hot

In [4]:
theta = np.array([10, 10])
zeta = 1e-4

In [5]:
chol = lambda a: cholesky(a, lower=True)
chol_solve = lambda l, b: cho_solve((l, True), b)

In [6]:
if n <= 10000:
    k = s_gaussian(x, x, theta)
    print(k.shape, k.nbytes / 1024**3)

(2000, 2000) 0.029802322387695312


In [7]:
if n <= 10000:
    l = chol(k + n * zeta * np.eye(n))
    print(l.shape, l.nbytes / 1024**3)

(2000, 2000) 0.029802322387695312


In [8]:
if n <= 10000:    
    k_q = s_gaussian(x, x_q, theta)
    print(k_q.shape, k_q.nbytes / 1024**3)

(2000, 10) 0.00014901161193847656


In [9]:
if n <= 10000:    
    result = np.dot(g.T, chol_solve(l, k_q))
    print(result)
    print(result.argmax(axis=1))

[[ 0.15  0.06  0.09  0.07  0.06  0.09  0.07  0.1   0.14  0.17]
 [ 0.06  0.08  0.19  0.16  0.13  0.11  0.07  0.14  0.13  0.11]
 [ 0.    0.07  0.16  0.12  0.14  0.14  0.1   0.14  0.07  0.15]
 [ 0.09  0.14  0.03  0.05  0.04  0.06  0.11  0.1   0.16  0.1 ]
 [ 0.08  0.09  0.1   0.08  0.12  0.15  0.15  0.16  0.06  0.07]
 [ 0.12  0.15  0.13  0.07  0.06  0.07  0.03  0.1   0.13  0.08]
 [ 0.21  0.09  0.01  0.15  0.19  0.1   0.12  0.07  0.07  0.11]
 [ 0.08  0.14  0.09  0.12  0.06  0.07  0.13  0.01  0.09  0.04]
 [ 0.1   0.07 -0.01  0.08  0.08  0.11  0.13  0.17  0.11  0.11]
 [ 0.08  0.1   0.18  0.08  0.09  0.09  0.08  0.01  0.04  0.05]]
[9 2 2 8 7 1 0 1 7 2]


In [10]:
rg = np.random.normal(loc=0, scale=1, size=(d, d))
rg_2 = np.random.normal(loc=0, scale=1, size=(d, d))
s = np.sum(rg_2 ** 2, axis=1)
q, r = np.linalg.qr(rg)
print(s.shape, q.shape)
print(rg.shape, rg.nbytes / 1024**3)
# w = rg / theta[1:]
w = (q * s).T / theta[1:]
print(w.shape, w.nbytes / 1024**3)
def phi(x):
    c = np.cos(np.dot(w, x.T)) / np.sqrt(d)
    s = np.sin(np.dot(w, x.T)) / np.sqrt(d)
    return np.concatenate((c, s), axis=0)

(3136,) (3136, 3136)
(3136, 3136) 0.073272705078125
(3136, 3136) 0.073272705078125


In [11]:
Q = phi(x).T
print(Q.shape, Q.nbytes / 1024**3)

(2000, 6272) 0.0934600830078125


In [12]:
g_new = np.dot(g.T, Q)

In [13]:
reg = n * zeta * np.eye(Q.shape[1]) / (theta[0] ** 2)
l_new = chol(np.dot(Q.T, Q) + reg)
print(l_new.shape, l_new.nbytes / 1024**3)

(6272, 6272) 0.2930908203125


In [14]:
Q_q = phi(x_q)
print(Q_q.shape, Q_q.nbytes / 1024**3)

(6272, 10) 0.0004673004150390625


In [15]:
result_new = np.dot(g_new, chol_solve(l_new, Q_q))

In [16]:
print(result_new)
print(result_new.argmax(axis=1))

[[-0.03 -0.2   0.26  0.01  0.11  0.2  -0.12  0.16 -0.38  0.05]
 [-0.07 -0.03  0.11  0.05 -0.27 -0.    0.23  0.08  0.   -0.43]
 [-0.34  0.1   0.24  0.38 -0.23 -0.12  0.14 -0.13  0.04 -0.03]
 [-0.31  0.07  0.32  0.07  0.16  0.19 -0.59 -0.27 -0.08 -0.35]
 [-0.02 -0.58  0.01 -0.12 -0.1   0.01  0.17 -0.28 -0.2  -0.13]
 [ 0.09 -0.16 -0.01 -0.09  0.34 -0.13  0.27 -0.23 -0.09  0.25]
 [ 0.2   0.1  -0.09  0.26 -0.09 -0.17 -0.2   0.3  -0.35  0.23]
 [-0.12  0.1  -0.27 -0.18 -0.    0.29  0.01 -0.2   0.42  0.08]
 [ 0.1   0.02  0.05  0.03  0.06  0.14 -0.2   0.04  0.01 -0.03]
 [-0.1   0.03  0.06 -0.19 -0.02 -0.33  0.12  0.28  0.03  0.  ]]
[2 6 3 2 6 4 7 8 5 7]


In [17]:
# if n <= 10000:    
#     k = np.dot(Q, Q.T)
#     l = chol(k + n * zeta * np.eye(n) / (theta[0] ** 2))
#     k_q = np.dot(Q, Q_q)
#     result = np.dot(g.T, chol_solve(l, k_q))
#     print(result)
#     print(result.argmax(axis=1))

In [1]:
import numpy as np
def generate_random_feature_square_weights(n_features):

    normals = np.random.normal(loc=0, scale=1, size=(n_features, n_features))
    q, r = np.linalg.qr(normals)
    normals = np.random.normal(loc=0, scale=1, size=(n_features, n_features))
    s = np.sum(normals ** 2, axis=1)
    g = (q * s).T
    return g

def generate_random_feature_weights(n_basis, n_features):

    n_repeats = int(n_basis / n_features) + 1
    gs = [generate_random_feature_square_weights(n_features) for i in range(n_repeats)]
    g_full = np.concatenate(tuple(gs), axis = 0)
    g = g_full[:n_basis]
    return g

In [2]:
generate_random_feature_square_weights(3)

array([[-0.95750523, -0.89879887,  0.64969324],
       [ 0.90984933, -2.92201005, -2.70145008],
       [ 4.23086587, -1.95143321,  3.53571512]])

In [3]:
generate_random_feature_weights(5, 3)

array([[-6.26915552,  1.50044422,  6.22054218],
       [-1.78391506, -2.00127379, -1.31513315],
       [ 3.68567034, -6.80498114,  5.35588928],
       [-1.02915641, -1.75699045,  0.55927114],
       [-0.02183948, -0.34647923, -1.12867786]])

In [4]:
generate_random_feature_weights(2, 3)

array([[-0.12454963,  2.67088687, -0.77299233],
       [ 0.75652838, -0.96835831, -3.46782335]])

In [5]:
generate_random_feature_weights(10, 3)

array([[-0.99011056,  0.52040181,  0.4163527 ],
       [-2.07534082, -1.61474601, -2.91699828],
       [-0.70177494, -3.11362701,  2.22288063],
       [-0.26972932,  0.18302112, -0.06751558],
       [-2.32424387, -4.38728257, -2.60752613],
       [-2.06657521, -1.45994614,  4.29848848],
       [-0.15025546,  2.73521163, -1.51022954],
       [-2.38620246, -1.29725763, -2.11208571],
       [-0.20566191,  0.08736627,  0.17869274],
       [-1.40683553,  0.07925304, -2.73136962]])